In [18]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

/Users/andrejezov/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
data = pd.read_csv('diamonds.txt', sep = '\t')
data.head()
X = data[["carat", "depth", "table", "x", "y", "z"]]
y = data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [20]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

rand_for = RandomForestRegressor(random_state=1)
rand_for.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [21]:
lin_reg_predictions = abs(y_test-lin_reg.predict(X_test))
rand_for_predictions = abs(y_test-rand_for.predict(X_test))

In [22]:
scipy.stats.ttest_ind(lin_reg_predictions, rand_for_predictions)

Ttest_indResult(statistic=6.1531297527436859, pvalue=7.7037322227091939e-10)

In [23]:
cm = CompareMeans(DescrStatsW(lin_reg_predictions), DescrStatsW(rand_for_predictions))

In [24]:
cm.tconfint_diff(usevar='unequal')

(59.124395353207831, 114.39972888765368)

In [25]:
T = (9.5-9.57)/(0.4/np.sqrt(160))

print 2 * (1.0-T)

6.42718872424


In [26]:
2.75/0.994426085089858

2.765414183349289

In [55]:
scipy.stats.norm.cdf(2.75)

0.99702023676494544

In [28]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [29]:
conf_interval_banner_a = proportion_confint(10, 
                                            34,
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(3, 
                                            16,
                                            method = 'wilson')
print conf_interval_banner_a
print conf_interval_banner_b

(0.16834630670422424, 0.46168909794714441)
(0.065915990714281392, 0.43008880961974144)


In [30]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [31]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [32]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [33]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind([1] * 10 + [0] * 24, [1] * 4 + [0] * 12), 'greater')

p-value: 0.372930


In [34]:
data = pd.read_csv('banknotes.txt', sep = '\t')
data.head()
X = data[["X1", "X2", "X3", "X4", "X5", "X6"]]
y = data["real"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=1)
print len(X_test)

50


In [35]:
from sklearn.linear_model import LogisticRegression

In [37]:
reg_1 = LogisticRegression()
reg_1.fit(X_train[["X1", "X2", "X3"]], y_train)

reg_2 = LogisticRegression()
reg_2.fit(X_train[["X4", "X5", "X6"]], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [60]:
predict1 = reg_1.predict(X_test[["X1", "X2", "X3"]])
predict2 = reg_2.predict(X_test[["X4", "X5", "X6"]])

In [43]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [61]:
proportions_diff_z_test(proportions_diff_z_stat_rel(y_test != predict1, y_test != predict2))

0.0032969384555543435

In [62]:
proportions_diff_confint_rel(y_test != predict1, y_test != predict2)

(0.059945206279614305, 0.30005479372038568)

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [58]:
z = (541.4- 525) / (100/(100**0.5))
print 1 - scipy.stats.norm.cdf(z)

0.0505025834741


Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки

In [59]:
z = (541.5- 525) / (100/(100**0.5))
print 1 - scipy.stats.norm.cdf(z)

0.0494714680336
